# TemplateDataProject - Basic Run

# 0. Initialization

In [ ]:
#@title Connect to gDrive & Load Files
if 'google.colab' in str(get_ipython()):
    print('Running on CoLab')
    PROJECT_FOLDER="/content/drive/MyDrive/workspace/template-data-project" #@param {"type":"string"}
    #@markdown - Connect to gDrive Folder
    ## Load this for local development version of library.. Loads from GoogleDrive (changes persist)
    from google.colab import drive, widgets, data_table; 
    drive.mount('/content/drive');
    %cd "$PROJECT_FOLDER"

    #@markdown - Install Requirements
    !pip install -qr requirements.txt

else:
    %cd /app
    print('Not running on CoLab')

    
#@markdown - Load .env
import re, os
lines = open('.env.yaml', 'r').read().split('\n')
for l in lines:
  segs = re.split(': *', l, 1)
  os.environ[segs[0]] = segs[1]

#@markdown - Import Utils .py FIles
import sys, os; from importlib import reload; sys.path.append(os.path.abspath("./src"))
from utils import notebook_helpers; reload(notebook_helpers)
!pwd

In [ ]:
#@title Imports
import pandas as pd
import plotly.express as px

# 1 Fetch and Parse Data

src/data/load_data.py


In [ ]:
from data import load_data; reload(load_data)
df = load_data.load_data()

# 2. Explore Data

In [ ]:
df

In [ ]:
px.parallel_coordinates(df, color='species_id')

In [ ]:
px.scatter_3d(df, 'sepal_length','petal_length','sepal_width',color='species')

# 3. Train Model
src/models/train_model.py

In [ ]:
from models import train_model; reload(train_model)
inputCols = ['sepal_length', 'sepal_width', 'petal_length', 'petal_width']
targetCol = ['species_id']

model = train_model.trainModelForDf(df, inputCols, targetCol)

# 4. Test & Visualize Results

In [ ]:
from models import predict_model; reload(predict_model)
from sklearn.metrics import confusion_matrix

## Create Test Set
df_test = df.sample(15)

## Predict
y_pred = predict_model.predict(model, df_test[inputCols])

## Display Results
pd.DataFrame(confusion_matrix(df_test[targetCol], y_pred))

# 5. Interactive App

In [ ]:
#@title Install jupyter_dash
!pip install -q jupyter-dash
import jupyter_dash
reload(jupyter_dash)

In [ ]:
import json
from dash import dcc, html
# from dash.dependencies import Input, Output, State


from src.dash.apps import dash_global

app = jupyter_dash.JupyterDash(__name__)

app.layout = html.Div([
    html.H1('Modeling', style={"textAlign": "center"}),
])

if __name__ == '__main__':
    app.run_server(mode='inline',
                    port=8501, 
                    host="0.0.0.0",
                    debug=True, **{'width': '1200px', 'height': '800px'}
      # assets_external_path="https://stackpath.bootstrapcdn.com/bootstrap/4.4.1/css/bootstrap.min.css"
    )

## External URL
Recommend Opening a Terminal (bottom-left) and run the following commends
- `ngrok http 8051 --log=stdout > ngrok.log &`
- `cat ngrok.log`
- `python src/dash/index.py`

In [ ]:
%%script false
#@title Skip.. Initialize NGROK
NGROK_AUTH="22ORETTuVHdmPUy8y4Vj4inqlQN_84tmp9jkzhKCJJ3kWGdK1" #@param
if 'NGROK_URL' not in globals():
  print('Loading ngrok..')
  !pip install pyngrok
  !ngrok authtoken NGROK_AUTH
  from pyngrok import ngrok
  NGROK_URL = ngrok.connect(addr=8051)
else : 
  print('Using Existing ngrok url..')

print(NGROK_URL)

In [ ]:
# %%script false
#@title Skip.. Multi Page Dash App
from src.dash.apps import data_overview, modeling

app3 = jupyter_dash.JupyterDash(__name__)

app3.layout = html.Div([
    dcc.Location(id='url', refresh=False),
    html.Div([
        dcc.Link('Data Overview | ', href='/apps/data_overview'),
        dcc.Link('Modeling', href='/apps/modeling'),
    ], className="row"),
    html.Div(id='page-content2', children=[])
])


@app3.callback(Output('page-content2', 'children'),
              [Input('url', 'pathname')])
def display_page(pathname):
    if pathname == '/apps/data_overview':
        return data_overview.layout
    if pathname == '/apps/modeling':
        return modeling.layout
    else:
        return "404 Page Error! Please choose a link"

app3.run_server(mode='external',port=8051, debug=True, **{'width': '200px', 'height': '200px'})
# app3._terminate_server_for_port('localhost', port=8051)